# introduction to rasterio
see instructions [here](https://rasterio.readthedocs.io/en/latest/quickstart.html#id2)


In [4]:
import rasterio

In [5]:
!ls data/downloaded

20200510_045002_24_2271_3B_AnalyticMS.tif
20200510_045004_44_2271_3B_AnalyticMS.tif
20200520_051901_1032_3B_AnalyticMS.tif
20200520_051902_1032_3B_AnalyticMS.tif


In [6]:
dataset = rasterio.open('data/downloaded/20200520_051901_1032_3B_AnalyticMS.tif')
print('name =', dataset.name)
print('mode =', dataset.mode)
print('closed?', dataset.closed)
print('numfiles:', dataset.count)
print('width:', dataset.width)
print('width:', dataset.height)
print('bounds', dataset.bounds)
print('transform', dataset.transform)
print('upper left corner', dataset.transform * (0, 0))
print('lower right corner', dataset.transform * (dataset.width, dataset.height))
print('coordinate system', dataset.crs)
print('indexes:', dataset.indexes)

name = data/downloaded/20200520_051901_1032_3B_AnalyticMS.tif
mode = r
closed? False
numfiles: 4
width: 8986
width: 4447
bounds BoundingBox(left=390192.0, bottom=2078097.0, right=417150.0, top=2091438.0)
transform | 3.00, 0.00, 390192.00|
| 0.00,-3.00, 2091438.00|
| 0.00, 0.00, 1.00|
upper left corner (390192.0, 2091438.0)
lower right corner (417150.0, 2078097.0)
coordinate system EPSG:32643
indexes: (1, 2, 3, 4)


In [7]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'uint16', 2: 'uint16', 3: 'uint16', 4: 'uint16'}

In [9]:
band1 = dataset.read(1)
print('center point', band1[dataset.height // 2, dataset.width // 2])
x, y = (dataset.bounds.left + 10000, dataset.bounds.top - 5000)
row, col = dataset.index(x, y)
print('value for the pixel 10 kilometers east and 5 kilometers south of the dataset’s upper left corner=', row, col, band1[row, col])
dataset.xy(dataset.height // 2, dataset.width // 2)

center point 6465
value for the pixel 10 kilometers east and 5 kilometers south of the dataset’s upper left corner= 1666 3333 7371


(403672.5, 2084767.5)

In [ ]:
# creating data
import numpy as np
x = np.linspace(-4.0, 4.0, 240)
y = np.linspace(-3.0, 3.0, 180)
X, Y = np.meshgrid(x, y)
Z1 = np.exp(-2 * np.log(2) * ((X - 0.5) ** 2 + (Y - 0.5) ** 2) / 1 ** 2)
Z2 = np.exp(-3 * np.log(2) * ((X + 0.5) ** 2 + (Y + 0.5) ** 2) / 2.5 ** 2)
Z = 10.0 * (Z2 - Z1)
from rasterio.transform import Affine
res = (x[-1] - x[0]) / 240.0
transform = Affine.translation(x[0] - res / 2, y[0] - res / 2) * Affine.scale(res, res)

new_dataset = rasterio.open(
    'data/img/new.tif',
    'w',
    driver='GTiff',
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs='+proj=latlong',
    transform=transform,
)
new_dataset.write(Z, 1)
new_dataset.close()


In [ ]:
# also possible
with rasterio.open(
    'data/img/new2.tif',
    'w',
    driver='GTiff',
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs='+proj=latlong',
    transform=transform,
) as dst:
    dst.write(Z, 1)


In [ ]:
import rasterio
src = rasterio.open('tests/data/RGB.byte.tif')
src.name
array = src.read(1)
array.shape
import matplotlib.pyplot as plt
plt.imshow(array, cmap='pink')
pyplot.show()

In [ ]:
array = src.read()
array.shape

In [ ]:
for i, dtype, nodataval in zip(src.indexes, src.dtypes, src.nodatavals):
    print(i, dtype, nodataval)


In [ ]:
src.close()


In [2]:
# Stockton, CA bounding box (created via geojson.io) 
geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
    [ 
      [-121.59290313720705, 37.93444993515032],
      [-121.27017974853516, 37.93444993515032],
      [-121.27017974853516, 38.065932950547484],
      [-121.59290313720705, 38.065932950547484],
      [-121.59290313720705, 37.93444993515032]
    ]
  ]
}


# Goes Great With GeoPandas
geojsonio.py integrates nicely with GeoPandas to display data in a GeoDataFrame.
Say you have a file containing the borders of all states called states.geojson. Each GeoJSON record has a property called 'Name'. Quickly display all the states whose names start with 'M'